In [1]:
import copy
import random

from get_data import get_data
from blocks import init_blocks, calc_factors, get_block_stat, get_block_penalty
from students import init_students, calc_weights, get_students_penalty, get_p0_students
from planner import get_try

In [2]:
blocks = get_data('data/blocks.json')

students_tmp = get_data('data/priorities.json')

N_blocks = len(blocks)
N_tries = 10

init_blocks(blocks)

students = init_students(students_tmp, blocks)

calc_factors(blocks)

calc_weights(students, N_blocks)

for block_nr in range(1, N_blocks+1):
    try_block = get_try(blocks[str(block_nr)], block_nr, students)
    block_penalty = get_block_penalty(try_block, block_nr, students)
    students_penalty = get_students_penalty(students, block_nr)

    for i in range(N_tries):
        try_block_next = get_try(blocks[str(block_nr)], block_nr, students)
        block_penalty_next = get_block_penalty(try_block_next, block_nr, students)
        students_penalty_next = get_students_penalty(students, block_nr)

        # if block_penalty_next < block_penalty:
        #     try_block = try_block_next
        #     block_penalty = block_penalty_next
            
        if students_penalty_next < students_penalty:
            try_block = try_block_next
            students_penalty = students_penalty_next
            
    print(block_nr, get_block_stat(try_block, block_nr, students)['total'], block_penalty, students_penalty)

    print(get_p0_students(try_block, block_nr, students))


1 {'total': 178, 'totp1': 62, 'totp2': 83, 'totp3': 23, 'totp0': 10} 417 954
[{'ID': '007', 'weight': 0}, {'ID': '042', 'weight': 3}, {'ID': '036', 'weight': 2}, {'ID': '103', 'weight': 2}, {'ID': '008', 'weight': 0}, {'ID': '087', 'weight': 2}, {'ID': '027', 'weight': 2}, {'ID': '121', 'weight': 1}, {'ID': '104', 'weight': 0}, {'ID': '136', 'weight': 3}]
2 {'total': 178, 'totp1': 95, 'totp2': 58, 'totp3': 17, 'totp0': 8} 393 503
[{'ID': '102', 'weight': 3}, {'ID': '171', 'weight': 1}, {'ID': '008', 'weight': 0}, {'ID': '147', 'weight': 3}, {'ID': '113', 'weight': 2}, {'ID': '121', 'weight': 1}, {'ID': '125', 'weight': 1}, {'ID': '007', 'weight': 0}]
3 {'total': 178, 'totp1': 77, 'totp2': 63, 'totp3': 29, 'totp0': 9} 431 588
[{'ID': '089', 'weight': 1}, {'ID': '059', 'weight': 1}, {'ID': '037', 'weight': 1}, {'ID': '008', 'weight': 0}, {'ID': '171', 'weight': 1}, {'ID': '084', 'weight': 1}, {'ID': '011', 'weight': 1}, {'ID': '019', 'weight': 1}, {'ID': '007', 'weight': 0}]
4 {'total': 

In [3]:
students

[{'ID': '000',
  'last_name': 'Abdallah',
  'first_name': 'Assma',
  'class': '3e',
  'block1': {'prio1': ['Universitätsspital Zürich'],
   'prio2': ['Varian', 'Hocoma'],
   'prio3': ['Weidmann', 'Sensirion'],
   'weight': 5,
   'choice': 'Sensirion'},
  'block2': {'prio1': ['Biologie (ETH)'],
   'prio2': ['Architektur (ETH)',
    'Umweltingenieurwissenschaften (ETH)',
    'Biodiversität in Zeit und Raum (UZH)'],
   'prio3': ['Autonome Agile Drohnen (UZH)'],
   'weight': 5},
  'block3': {'prio1': ['Medizin studieren an der ETH'],
   'prio2': ['Chemie (ETH)', 'Pharmazeutische Wissenschaften (ETH)'],
   'prio3': [],
   'weight': 3},
  'block4': {'prio1': ['Krebsforschung (UZH)'],
   'prio2': ['Mathematik (ETH)'],
   'prio3': ['Aktuelle Forschung am CERN (UZH)'],
   'weight': 3},
  'block5': {'prio1': ['Woher kommst du wirklich? Frag deine DNA!'],
   'prio2': ['Fourier-Reihen',
    'Astronomie',
    'Entsorgung, Recycling und Abwasserreinigung in Zürich',
    'Praktische Molekulargenetik: